<a href="https://colab.research.google.com/github/UdaraChamidu/Large-Language-Models/blob/main/FineTune_Llama_3_2_1B_Instruct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install unsloth library

In [24]:
# ulsoth - a python packege that is used to finetuned

%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# Use the Language Model

In [25]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-1B-Instruct",  # Confirm this exists publicly
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True
)


==((====))==  Unsloth 2025.6.2: Fast Llama patching. Transformers: 4.52.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [26]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 8, # quantized bit
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 8,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

In [27]:
{"role": "system", "content": "You are a knowledgeable chatbot on wildlife and wildlife forest reserves in Sri Lanka."}
{"role": "user", "content": "How many wild life parks in Sri Lanka?"}
{"role": "assistant", "content": "There are 26 wild life parks in Sri Lanka."}

{'role': 'assistant', 'content': 'There are 26 wild life parks in Sri Lanka.'}

# Load the dataset

In [28]:
import json
from unsloth.chat_templates import get_chat_template, standardize_sharegpt
from datasets import Dataset

# Load the JSON file
file_path = 'eye_disease_dataset.json'
with open(file_path, 'r') as f:
    data = json.load(f)

# Group the messages into conversations (pair user and assistant messages)
conversations = []
current_convo = []
for message in data:
    current_convo.append(message)
    if message["role"] == "assistant":  # End of a conversation turn (user question + assistant answer)
        conversations.append(current_convo)
        current_convo = []

# Convert list of conversations into a Hugging Face Dataset
dataset = Dataset.from_dict({"conversations": conversations})

# Apply the standardize_sharegpt function
dataset = standardize_sharegpt(dataset)

# Define the tokenizer with a chat template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

# Define the formatting function for the prompts
def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }

# Map over the dataset
dataset = dataset.map(formatting_prompts_func, batched=True)

# Your dataset is now formatted and ready

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/346 [00:00<?, ? examples/s]

Map:   0%|          | 0/346 [00:00<?, ? examples/s]

In [29]:
dataset[5]["conversations"]

[{'content': 'What are the stages of diabetic retinopathy?', 'role': 'user'},
 {'content': 'Non-proliferative diabetic retinopathy (NPDR) and proliferative diabetic retinopathy (PDR).',
  'role': 'assistant'}]

In [30]:
dataset[5]["text"]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhat are the stages of diabetic retinopathy?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nNon-proliferative diabetic retinopathy (NPDR) and proliferative diabetic retinopathy (PDR).<|eot_id|>'

# Definning the training model

In [31]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 50, # Set this for 1 full training run.
        #max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/346 [00:00<?, ? examples/s]

In [32]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map (num_proc=2):   0%|          | 0/346 [00:00<?, ? examples/s]

# Train the dataset

In [33]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 346 | Num Epochs = 50 | Total steps = 2,200
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 5,636,096/1,000,000,000 (0.56% trained)


Step,Training Loss
1,2.975300
2,2.921000
3,2.602900
4,2.599000
5,2.029500
6,2.676100
7,2.105700
8,1.473500
9,2.239100
10,1.736100


# Testing

In [34]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Tell me diognosis of diabetic retinopathy?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                         temperature = 0.7, min_p = 0.1, top_p=0.9)
tokenizer.batch_decode(outputs)

['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nTell me diognosis of diabetic retinopathy?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nA complication of diabetes affecting the retina, leading to vision loss.<|eot_id|>']

In [35]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Tell me diognosis of diabetic retinopathy?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 0.7, min_p = 0.1, top_p=0.9)

A complication of diabetes leading to vision loss through retinal damage.<|eot_id|>


# Saving

In [38]:
model.save_pretrained_gguf("Llama-3.2-1B-Instruct_FineTuned", tokenizer, quantization_method = "q8_0")

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 5.38 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 16/16 [00:00<00:00, 37.49it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving Llama-3.2-1B-Instruct_FineTuned/pytorch_model.bin...
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q8_0'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: [1] Converting model at Llama-3.2-1B-Instruct_FineTuned into q8_0 GGUF format.
The output location will be /content/Llama-3.2-1B-Instruct_FineTuned/unsloth.Q8_0.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: Llama-3.2-1B-Instruct_FineTuned
INFO:hf-to-gguf:Model architecture: LlamaForCausalLM
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 -->

In [39]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
